In [1]:
import heeps
import numpy as np
from astropy.io import fits

### ideal

In [2]:
conf = dict(
    mode = 'ELT',
    dir_output = 'instant_psf/ideal',
    add_phase = False,
#    f_phase = 'wavefront/cfull/cube_Cfull_20220512_3600s_300ms_0piston_meters_scao_only_L_285.fits',
    nframes = 1,
    hfov = 1.065,
    ngrid = 2048,
)
conf = heeps.config.read_config(**conf)
conf['band_specs']['L']['pscale'] = 5.47/2
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf = heeps.wavefront.propagate(wf, **conf)[0]
peak = np.max(psf)
print('peak = %.4f'%peak)
psf /= peak
fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_ELT.fits', psf, overwrite=True)

   2022-06-29 15:28:48, e2e simulation using 12 cores
   2022-06-29 15:28:51, completed in 3.18 seconds
peak = 0.0122


In [3]:
verbose = False
for conf['mode'] in ['RAVC', 'APPIMG', 'CVC', 'CLC']:
    if 'RAVC' in conf['mode']:
        conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
    elif 'CVC' in conf['mode']:
        conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
    elif 'CLC' in conf['mode']:
        conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
    conf = heeps.config.update_config(**conf, verbose=verbose)
    wf = heeps.pupil.pupil(**conf)
    psf = heeps.wavefront.propagate(wf, verbose=verbose,  **conf)[0]
    psf /= peak
    fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_%s.fits'%conf['mode'], psf, overwrite=True)

   2022-06-29 15:28:54, e2e simulation using 12 cores
   2022-06-29 15:29:04, completed in 10.4 seconds
   2022-06-29 15:29:05, e2e simulation using 12 cores
   2022-06-29 15:29:11, completed in 6.19 seconds
   2022-06-29 15:29:14, e2e simulation using 12 cores
   2022-06-29 15:29:26, completed in 12.69 seconds
   2022-06-29 15:29:29, e2e simulation using 12 cores
   2022-06-29 15:29:45, completed in 16.06 seconds


### scao only

In [4]:
conf = dict(
    mode = 'ELT',
    dir_output = 'instant_psf/scao_only',
    add_phase = True,
    nframes = 1,
    hfov = 1.065,
    ngrid = 2048,
)
conf = heeps.config.read_config(**conf)
conf['band_specs']['L']['pscale'] = 5.47/2
conf = heeps.config.update_config(**conf)
wf = heeps.pupil.pupil(**conf)
psf = heeps.wavefront.propagate(wf, **conf)[0]
peak = np.max(psf)
print('peak = %.4f'%peak)
psf /= peak
fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_ELT.fits', psf, overwrite=True)

   2022-06-29 15:29:46, e2e simulation using 12 cores
   2022-06-29 15:29:51, completed in 5.08 seconds
peak = 0.0120


In [5]:
verbose = False
for conf['mode'] in ['RAVC', 'APPIMG', 'CVC', 'CLC']:
    if 'RAVC' in conf['mode']:
        conf.update(ls_dRext=0.0477, ls_dRint=0.04, ls_dRspi=0.0249)
    elif 'CVC' in conf['mode']:
        conf.update(ls_dRext=0.0209, ls_dRint=0.09, ls_dRspi=0.0245)
    elif 'CLC' in conf['mode']:
        conf.update(ls_dRext=0.1095, ls_dRint=0.0491, ls_dRspi=0.0317)
    conf = heeps.config.update_config(**conf, verbose=verbose)
    wf = heeps.pupil.pupil(**conf)
    psf = heeps.wavefront.propagate(wf, verbose=verbose,  **conf)[0]
    psf /= peak
    fits.writeto(conf['dir_output'] + 'onaxis_PSF_L_%s.fits'%conf['mode'], psf, overwrite=True)

   2022-06-29 15:29:56, e2e simulation using 12 cores
   2022-06-29 15:30:15, completed in 18.75 seconds
   2022-06-29 15:30:16, e2e simulation using 12 cores
   2022-06-29 15:30:24, completed in 8.28 seconds
   2022-06-29 15:30:27, e2e simulation using 12 cores
   2022-06-29 15:30:45, completed in 17.3 seconds
   2022-06-29 15:30:48, e2e simulation using 12 cores
   2022-06-29 15:31:10, completed in 21.72 seconds
